In [1]:
import pandas as pd
T = pd.read_csv('../data/train.csv.gz')

In [2]:
# Add more fields

T['DateTime'] = pd.to_datetime(T['DateTime'])
t = T.DateTime
T['dt_year'] = t.dt.year
T['dt_month'] = t.dt.month
T['dt_dayofweek'] = t.dt.dayofweek

In [3]:
# convert ages to the same unit (days)
ages_day = []
for val in T.AgeuponOutcome:
    if not isinstance(val, str) and np.isnan(val):
        ages_day.append(val)
        continue
    num, unit = val.split()
    num = float(num)
    if 'day' in unit:
        ages_day.append(num)
    elif 'month' in unit:
        ages_day.append(num*30)
    elif 'week' in unit:
        ages_day.append(num*7)
    elif 'year' in unit:
        ages_day.append(num*365)
    else:
        raise ValueError('Unknown unit ' + unit )

T['AgeuponOutcome_days'] = ages_day

# make desexed column and male/female column

#print(T.SexuponOutcome.unique())
sex = T.SexuponOutcome.copy()
sex[np.in1d(sex, ('Neutered Male', 'Intact Male'))] = 'Male'
sex[np.in1d(sex, ('Spayed Female', 'Intact Female'))] = 'Female'
sex[sex == 'Unknown'] = np.nan
#print(sex.unique())
T['sex'] = sex

desexed = T.SexuponOutcome.copy()
desexed[np.in1d(T['SexuponOutcome'], ('Neutered Male','Spayed Female'))] = 'Yes'
desexed[np.in1d(T['SexuponOutcome'], ('Intact Male','Intact Female'))] = 'No'
desexed[T['SexuponOutcome'] == 'Unknown'] = np.nan
#print(desexed.unique())
T['desexed'] = desexed

In [4]:
# whether it's a mixed breed

T['Breed_is_mix'] = np.array(['mix' in val.lower() for val in T['Breed']])

In [5]:
# make a unique list of breeds, removing mix words (to assign dog sizes)
breeds = dict(dog=set([]), cat=set([]))
for i,row in T.iterrows():
    val = row['Breed']
    key = row['AnimalType'].lower()
    val = val.lower()
    if val.endswith(' mix'):
        val = val[:-4]
    items = val.split('/')
    for item in items:
        breeds[key].add(item)
# now add size info to dog breeds (see below)

In [6]:
# Here I add a column giving the size information for each breed of dog. 
# Sizes taken from http://www.dogchannel.com/dog-breeds

s = """affenpinscher  , s
afghan hound   , m 
airedale terrier, m
akita         ,l
alaskan husky  , u
alaskan malamute   ,l
american bulldog    , l
american eskimo     , s 
american foxhound   , l
american pit bull terrier, m
american staffordshire terrier, m
anatol shepherd, l
australian cattle dog, m
australian kelpie, m
australian shepherd,  m
australian terrier,   s
basenji, m
basset hound, m
beagle, m
bearded collie, m
beauceron, l
bedlington terr, s
belgian malinois, l
belgian sheepdog, m
belgian tervuren, m
bernese mountain dog, l
bichon frise, s
black, u
black mouth cur, u
bloodhound, l
blue lacy, u
bluetick hound, m
boerboel, l
border collie, m
border terrier, s
borzoi, l
boston terrier, s
boxer, m
boykin span, m
brittany, m
bruss griffon, s
bull terrier, m
bull terrier miniature, s
bulldog, m
bullmastiff, l
cairn terrier, s
canaan dog, m
cane corso, l
cardigan welsh corgi, m
carolina dog, m
catahoula, u
cavalier span, s
chesa bay retr, m
chihuahua longhair, s
chihuahua shorthair, s
chinese crested, s
chinese sharpei, m
chow chow, m
cocker spaniel, m
collie rough, m
collie smooth, m
dachshund, s
dachshund longhair, s
dachshund wirehair, s
dalmatian, m
doberman pinsch, l
dogo argentino, l
dogue de bordeaux, l
dutch shepherd, m
english bulldog, m
english cocker spaniel, m
english coonhound, u
english foxhound, m
english pointer, u
english setter, m
english shepherd, m
english springer spaniel, m
entlebucher, m
feist, u
field spaniel, m
finnish spitz, m
flat coat retriever, m
french bulldog, s
german pinscher, m
german shepherd, l
german shorthair pointer, m
german wirehaired pointer, m
glen of imaal, m
golden retriever, m
great dane, l
great pyrenees, l
greater swiss mountain dog, l
greyhound, l
harrier, m
havanese, s
hovawart, l
ibizan hound, m
irish setter, l
irish terrier, m
irish wolfhound, l
italian greyhound, s
jack russell terrier, s
japanese chin, s
jindo, u
keeshond, m
kuvasz, l
labrador retriever, m
landseer, u
leonberger, l
lhasa apso, s
lowchen, s
maltese, s
manchester terrier, s
mastiff, l
mexican hairless, u
miniature pinscher, s
miniature poodle, s
miniature schnauzer, s
neapolitan mastiff, l
newfoundland, l
norfolk terrier, s
norwegian elkhound, m
norwich terrier, s
nova scotia duck tolling retriever, m
old english bulldog, l
old english sheepdog, l
otterhound, l
papillon, s
parson russell terrier, s
patterdale terr, s
pbgv, u
pekingese, s
pembroke welsh corgi, m
pharaoh hound, m
picardy sheepdog, u
pit bull, m
plott hound, m
podengo pequeno, u
pointer, m
pomeranian, s
port water dog, m
presa canario, u
pug, s
queensland heeler, m
rat terrier, s
redbone hound, m
rhod ridgeback, l
rottweiler, m
saluki, m
samoyed, m
schipperke, s
schnauzer giant, l
scottish terrier, s
sealyham terr, s
shetland sheepdog, s
shiba inu, s
shih tzu, s
siberian husky, m
silky terrier, s
skye terrier, m
smooth fox terrier, s
soft coated wheaten terrier, s
spanish mastiff, l
spanish water dog, m
spinone italiano, l
st. bernard rough coat, l
st. bernard smooth coat, l
staffordshire, m
standard poodle, m
standard schnauzer, m
swedish vallhund, m
swiss hound, u
tan hound, u
tibetan spaniel, s
tibetan terrier, s
toy fox terrier, s
toy poodle, s
treeing cur, u
treeing tennesse brindle, u
treeing walker coonhound, m
unknown, u
vizsla, l
weimaraner, l
welsh springer spaniel, m
welsh terrier, s
west highland, s
whippet, m
wire hair fox terrier, s
wirehaired pointing griffon, m
yorkshire, s
yorkshire terrier, s"""

size_from_breed = {}
for row in s.split('\n'):
    #print(row)
    breed, size  = row.split(',')
    size_from_breed[breed.strip()] = size.strip()

sizes = []
for i,row in T.iterrows():
    if row['AnimalType'] != 'Dog':
        sizes.append('u')
        continue
    else:
        breed = row['Breed']
    breed = breed.lower()
    if breed.endswith(' mix'):
        breed = breed[:-4]
    breed = breed.split('/')[0]
    if breed in size_from_breed:  
        sizes.append(size_from_breed[breed])
    else:
        sizes.append('u')

T['dogsize'] = sizes

In [7]:
# Now let's take a look at the table
T

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,dt_year,dt_month,dt_dayofweek,AgeuponOutcome_days,sex,desexed,Breed_is_mix,dogsize
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,2014,2,2,365.0,Male,Yes,True,s
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,2013,10,6,365.0,Female,Yes,True,u
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,2015,1,5,730.0,Male,Yes,True,m
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,2014,7,4,21.0,Male,No,True,u
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,2013,11,4,730.0,Male,Yes,False,s
5,A677334,Elsa,2014-04-25 13:04:00,Transfer,Partner,Dog,Intact Female,1 month,Cairn Terrier/Chihuahua Shorthair,Black/Tan,2014,4,4,30.0,Female,No,False,s
6,A699218,Jimmy,2015-03-28 13:11:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Tabby,2015,3,5,21.0,Male,No,True,u
7,A701489,NaN,2015-04-30 17:02:00,Transfer,Partner,Cat,Unknown,3 weeks,Domestic Shorthair Mix,Brown Tabby,2015,4,3,21.0,NaN,NaN,True,u
8,A671784,Lucy,2014-02-04 17:17:00,Adoption,NaN,Dog,Spayed Female,5 months,American Pit Bull Terrier Mix,Red/White,2014,2,1,150.0,Female,Yes,True,m
9,A677747,NaN,2014-05-03 07:48:00,Adoption,Offsite,Dog,Spayed Female,1 year,Cairn Terrier,White,2014,5,5,365.0,Female,Yes,False,s


In [8]:
T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26729 entries, 0 to 26728
Data columns (total 18 columns):
AnimalID               26729 non-null object
Name                   19038 non-null object
DateTime               26729 non-null datetime64[ns]
OutcomeType            26729 non-null object
OutcomeSubtype         13117 non-null object
AnimalType             26729 non-null object
SexuponOutcome         26728 non-null object
AgeuponOutcome         26711 non-null object
Breed                  26729 non-null object
Color                  26729 non-null object
dt_year                26729 non-null int64
dt_month               26729 non-null int64
dt_dayofweek           26729 non-null int64
AgeuponOutcome_days    26711 non-null float64
sex                    25635 non-null object
desexed                25635 non-null object
Breed_is_mix           26729 non-null bool
dogsize                26729 non-null object
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3), object(12)
memory u

In [9]:
for name in T.columns:
    print('\n### ' + name + '\n', T[name].describe(), sep='')


### AnimalID
count       26729
unique      26729
top       A694918
freq            1
Name: AnimalID, dtype: object

### Name
count     19038
unique     6374
top         Max
freq        136
Name: Name, dtype: object

### DateTime
count                   26729
unique                  22918
top       2015-08-11 00:00:00
freq                       19
first     2013-10-01 09:31:00
last      2016-02-21 19:17:00
Name: DateTime, dtype: object

### OutcomeType
count        26729
unique           5
top       Adoption
freq         10769
Name: OutcomeType, dtype: object

### OutcomeSubtype
count       13117
unique         16
top       Partner
freq         7816
Name: OutcomeSubtype, dtype: object

### AnimalType
count     26729
unique        2
top         Dog
freq      15595
Name: AnimalType, dtype: object

### SexuponOutcome
count             26728
unique                5
top       Neutered Male
freq               9779
Name: SexuponOutcome, dtype: object

### AgeuponOutcome
count      26711
uniqu

/Users/ncrighton/anaconda/envs/py3/lib/python3.5/site-packages/numpy/lib/function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [10]:
# use a pivot table to examine which parameters have the biggest effect on the outcome. 
from pivottablejs import pivot_ui
pivot_ui(T)

The outcome depends strongly on the age (AgeuponOutcome_days), whether the animal has been desexed, and on the animal type (dog or cat). It doesn't depend much on the sex (male/female), or on the dog size. There doesn't seem to be a big change in outcome with day of the week, month or year of the outcome.

Below are plots showing how the outcome changes with the animal type (dog or cat), with the desexed status (intact vs spayed/neutered) and the age of the animal, which seem to have the biggest influence on the outcome. They are all screenshots from the pivottablejs app above; it was simpler to just take these screenshots instead of working out how to reproduce the plots in pandas.

## Outcome vs type

The outcome vs the animal type. Cats are much more likely to be transferred than dogs, and less likely to be returned to their owner.

<img src="img/outcome_vs_type.png" style="width: 700px"/>

## Outcome vs age

There are two plots here - the first shows the total number of animals vs age, to give you a feel for the age distribution. The second shows the fraction of animals falling into each outcome vs age. Animals less than a month old are usually transferred. Animals older than one month are typically transferred, adopted, or returned to their owner. As the animals get older (> 3650 days, which is > 10 years), they are more likely to be euthanased.

<img src="img/outcome_vs_age_tot.png" style="width: 700px"/>
<img src="img/outcome_vs_age.png" style="width: 700px"/>

## Outcome vs desexed

This shows the outcome vs whether the dog or cat has been desexed. Desexed animals are more likely to be adopted or returned to their owner. Non-desexed animals are more likely to be transferred or euthanased.

<img src="img/outcome_vs_desexed.png" style="width: 700px"/>


## Outcome vs size

There isn't much change in outcome with dog size ('u' here means unknown - all the cats have size=u). Maybe medium-sized dogs are more likely to be euthanased?

<img src="img/outcome_vs_size.png" style="width: 700px"/>


In [ ]:
# write out the new training table which has all the new columns added.
T.to_csv('../data/train_prep.csv')
